In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from src.main import data_normalization

## Loading datasets 

Runtime: roughly 3 minutes.

In [ ]:
# Load the normalized data
data, embeddings = data_normalization()

# Extract relevant dataframes
news_df = data["news"]
behaviors_df = data["behaviors"]

behaviors_df.head()

The mistake i did last time was to assume "Clicked" and "News ID" was already there - I assume that i planned to make them, but wrote the draft and forgot to actually do it. 

"Impressions" will now be split on "-" into "News ID" and "Clicked". I assume this is correct, feel free to correct me. Then "User ID" will be added through the old dataset.

Runtime: 20 minutes - 1 hour

In [ ]:
# Ensure required columns exist
if "Impressions" in behaviors_df.columns:
    # Split "Impressions" to extract News ID and Clicked values
    impressions_expanded = behaviors_df["Impressions"].astype(str).str.split(" ", expand=True).stack().reset_index(level=1, drop=True)
    impressions_df = impressions_expanded.astype(str).str.split("-", expand=True).rename(columns={0: "News ID", 1: "Clicked"})

    # Clean the "Clicked" column by stripping unwanted characters
    impressions_df["Clicked"] = impressions_df["Clicked"].str.replace(r"[^\d]", "", regex=True).astype(int)

    # Convert "clicked" to type int
    impressions_df["Clicked"] = impressions_df["Clicked"].astype(int)

    # Removing the ' that appears in the beginning of every News ID
    impressions_df["News ID"] = impressions_df["News ID"].str.strip("'")  # Remove leading quotes

    # Add back User ID from the original dataframe
    impressions_df = impressions_df.join(behaviors_df["User ID"], how="left")
    
    print(impressions_df.head())

else:
    print("Impressions is missing.")

This is perhaps something Jolan can implement in the normalization?

In [ ]:
print(f"Unique Users: {impressions_df['User ID'].nunique()}")
print(f"Unique News: {impressions_df['News ID'].nunique()}")

There are alot of unique values, making the matrix incredibly large. I will make a sparse matrix instead.

## Creating matrix and applying filters

In [ ]:
from scipy.sparse import coo_matrix

In [ ]:
print(impressions_df.head())
print(impressions_df.dtypes)
print(impressions_df["User ID"].unique()[:10])  # Show first 10 unique User IDs
print(impressions_df["News ID"].unique()[:10])  # Show first 10 unique News IDs


In [ ]:
# Convert User ID and News ID to categorical codes (numeric indices)
impressions_df["User Index"] = impressions_df["User ID"].astype("category").cat.codes
impressions_df["News Index"] = impressions_df["News ID"].astype("category").cat.codes

Then a User-Item interaction matrix will be made - first it will check for the necessary columns. A sparse matrix only stores entries where an item was interacted with, ignoring all those where the user didn't clikc on the article.

Interaction matrices: these matrices are tables that represent user behaviour - what each user has interacted with.

In [ ]:
# Create User-Item Interaction matrix
if "User ID" in impressions_df.columns and "News ID" in impressions_df.columns:

    # Ensure there are no duplicate User ID and News ID pair
    impressions_df = impressions_df.groupby(["User ID", "News ID"])["Clicked"].max().reset_index()
    
    # Create sparse interaction matrix
    interaction_matrix = coo_matrix((impressions_df["Clicked"], (impressions_df["User ID"], impressions_df["News ID"])), shape=(len(user_mapping), len(news_mapping)))

    print(f"Sparse Matrix Shape: {interaction_matrix.shape}")
    print(f"Non-zero interactions: {interaction_matrix.nnz}")  # Check sparsity

else:
    interaction_matrix = pd.DataFrame()
    print("Missing columns.")

Collaborative filtering is applied, given that the matrix isn't empty. This uses SVD.

Collaborative filtering: finds similar users based on their history, and recommends similar items to similar users.

In [ ]:
# Apply collaborative filtering
if not interaction_matrix.empty:
    U, sigma, Vt = svds(interaction_matrix, k=50) # Reduce dimensions
    sigma = np.diag(sigma)
    predicted_ratings = np.dot(np.dot(U, sigma), Vt)
    predicted_df = pd.DataFrame(predicted_ratings, index=interaction_matrix.index, columns=interaction_matrix.columns)
    print(predicted_df.head())

else:
    predicted_df = pd.DataFrame()

Then, content-based filtering is applied. Here we will be using TF-IDF (cosine similarity), but this may be changed in the future - perhaps into a word embedding method.

Content-based filtering: recommends items by analyzing the attributes of items, and matching them with a user's preferences/past interactions. Measures similarity between items.

In [ ]:
# Apply content-based filtering
vectorizer = TfidfVectorizer(stop_words="english")
tfidf_matrix = vectorizer.fit_transform(news_df["Title"] + " " + news_df["Abstract"])
similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Store results
news_similarity_df = pd.DataFrame(similarity_matrix, index=news_df["News ID"], columns=news_df["News ID"])

In [ ]:
# Hybrid approach, combining scores
def hybrid_recommendations(user_id, top_n=5, alpha=0.5):
    if user_id not in predicted_df.index:
        return [] # No recommendations for uknown/new users
    
    # Normalize collaborative scores
    user_ratings = predicted_df.loc[user_id].copy()
    user_ratings = (user_ratings - user_ratings.min()) / (user_ratings.max() - user_ratings.min()) # Normalize

    # Compute content-based scores
    content_scores = news_similarity_df[user_ratings.index].dot(user_ratings.fillna(0))
    content_scores = (content_scores - content_scores.min()) / (content_scores.max() - content_scores.min()) # Normalize

    # Combination of both scores, using weights
    final_scores = alpha * user_ratings + (1 - alpha) * content_scores
    return final_scores.nlargest(top_n).index.tolist()

In [ ]:
# Testing the recommender
if not interaction_matrix.empty:
    user_id = interaction_matrix.index[0] # Pick a sample user
    recommendations = hybrid_recommendations(user_id)
    print("Recommended articles: ", recommendations)

else:
    print("No valid interactions.")